In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [12]:
import pandas as pd
from datetime import datetime


pos = 1
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,dois_riachos


In [13]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [14]:
def people():
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")

#     children = nav.find_element(By.ID, 'maxAge')
#     nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
#     time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

    pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
    time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [15]:
def get_table():
    
    list_links = []
    people()
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                print(ElementClickInterceptedException)
                
            if NoSuchElementException: 
                print(NoSuchElementException)
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [16]:
def get_indivivual_info():
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, has, dm, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], [], [], []

    
    individual = get_table()
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        linha_dois = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[1]').text
        linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
        
        linha_dois_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
        
        if(linha_dois.upper() == "CPF"):
            cpf.append(linha_dois_text)
            
        elif(linha_tres.upper() == "CPF"):
            cpf.append(linha_tres_text)
            
        else:
            cpf.append("--")
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        
        if i_cns[0].upper() != 'CNS':
            cns.append('--')
        else:
            cns.append(i_cns[1])
        
        i_has = "sim"
        has.append(i_has)
        
        i_dm = "não"
        dm.append(i_dm)
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
        
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        try:
            linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
            linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
            if(linha_tres.upper() == 'PROFISSIONAL'):
                profissional.append(linha_tres_text)
        except:
            continue
        try:
            linha_quatro = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[1]').text
            linha_quatro_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
            if(linha_quatro.upper() == 'PROFISSIONAL'):
                profissional.append(linha_quatro_text)
        except:
            continue
        
        try:
            linha_cinco = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[1]').text
            linha_cinco_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[2]').text
            if(linha_cinco.upper() == 'PROFISSIONAL'):
                profissional.append(linha_cinco_text)
        except:
            continue
        
        try:
            linha_seis = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[1]').text
            linha_seis_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[2]').text
            if(linha_seis.upper() == 'PROFISSIONAL'):
                profissional.append(linha_seis_text)
            
        except:
            continue
            
        time.sleep(1)

    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "has":has, "dm":dm, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')
    
    nav.quit()
    
    return df_individual

# indi = get_indivivual_info()

In [17]:
pregnant = get_indivivual_info()

Extração iniciada as: 2024-01-26 17:35:24.826619
Iniciando lista de links: 2024-01-26 17:35:41.654781
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
acabaram as paginas
lista de links finalizada as: 2024-01-26 17:35:41.840210
Iniciando individuos : 2024-01-26 17:35:41.840228
Finalizando individuos : 2024-01-26 17:36:44.265775
Extração finalizada as: 2024-01-26 17:36:44.268839


In [18]:
pregnant = pregnant[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'link']]
display(pregnant)


,cpf,name,b_date,sex,race,m_name,link
0,10459655450,ALINE SANTOS DA SILVA,19/08/1992,FEMININO,PARDA,MARIA DO SOCORRO SANTOS DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
1,--,CRISTIANE ISABEL BARROS BEZERRA,28/10/1986,FEMININO,PARDA,IZABEL CRISTINA DE BARROS,https://afogadosdaingazeira.ephealth.com.br/po...
2,14035864463,DARLENE CARVALHO LEANDRO DA SILVA,28/12/2003,FEMININO,BRANCA,ROSE MERE CARVALHO LEANDRO DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
3,07607724443,EDJANEIDE ALVES DA SILVA,10/09/1988,FEMININO,PARDA,MARIA DAS GRAÇAS FEITOSA ALVES,https://afogadosdaingazeira.ephealth.com.br/po...
4,--,FERNANDA BEZERRA DA SILVA,07/02/2003,FEMININO,BRANCA,NEILDA BEZERRA FIRMINO,https://afogadosdaingazeira.ephealth.com.br/po...
5,--,JUSSARA MIKAELE LOPES DE ALCANTARA,23/10/1991,FEMININO,BRANCA,JOSEFA LOPES DE ALCANTARA,https://afogadosdaingazeira.ephealth.com.br/po...
6,12658624433,LUZIA APARECIDA DOS SANTOS,01/12/1997,FEMININO,BRANCA,MARIA MARIANO DA SILVA SANTOS,https://afogadosdaingazeira.ephealth.com.br/po...
7,17236077461,MAIARA BEZERRA DE CARVALHO,15/10/2006,FEMININO,PARDA,MARIA DANIELA BEZERRA DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
8,72059143454,MARIA APARECIDA DOS SANTOS SILVA,19/03/1983,FEMININO,BRANCA,MARIA ELIDIA DOS SANTOS,https://afogadosdaingazeira.ephealth.com.br/po...
9,11226875408,MARIA DAS DORES DA SILVA,29/01/1993,FEMININO,PARDA,ODETE MARIA DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...


In [21]:
pregnant = pregnant[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'link']]
pregnant.to_excel(f'{data_e_hora}_{name}_pregnant.xlsx', index=False)